In [37]:
'''
2017 National Emissions Inventory - Point Sources
Data Downloaded at https://www.epa.gov/air-emissions-inventories/2017-national-emissions-inventory-nei-data#datas

The purpose of this notebook is to calculate the total emissions within a select radius of a monitor and an industrial facility
This type of analysis is required to demonstrate the a given monitor is conservative/representative of the air surrounding the industrial facility

Required Inputs within the notebook include:
- User Specified Radius (km)
- Lat/Long location of ambient air monitor
- Lat/Long location of industrial facility
'''
import numpy as np
import pandas as pd
#input_file = "../2017neiJan_facility_process_byregions/point_12345.csv" #Use this for locations within EPA Regions 1, 2, 3, 4, 5
input_file = "../2017neiJan_facility_process_byregions/point_678910.csv" #Use this for locations within EPA Regions 6, 7, 8, 9, 10

df = pd.read_csv(input_file)

C:\Users\steven\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (3,8,9,10,19,22,26,28,30,31,32,36,37,38,39,47,48,49) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [38]:
#Create your Filters
state_filt = (df["state"] == "TX") #Search within Texas, which is in Region 6
poll_filt = (df["pollutant_code"] == "PM25-PRI") #Search for Primary PM2.5 emissions
sortby = state_filt & poll_filt

In [39]:
# Want the Emissions Totals centered around a monitor and the industrial facility. Enter the locations in (Lat, Long) here as well as the search radius.
search_radius = 15 #this is km
Mon_Location = (29.515077, -98.620218)
Site_Location = (29.000000, -98.000000)

In [40]:
# These are the column headers, also identified in the provided PDF file
df.columns

Index(['region', 'state', 'stfips', 'tribal_name', 'fips', 'county',
       'eis_facility_id', 'program_system_code', 'agency_facility_id',
       'tri_facility_id', 'company_name', 'site_name', 'naics_code',
       'naics_description', 'facility_source_type', 'site_latitude',
       'site_longitude', 'address', 'city', 'zip_code', 'postal_abbreviation',
       'eis_unit_id', 'agency_unit_id', 'unit_type', 'unit_description',
       'design_capacity', 'design_capacity_uom', 'eis_process_id',
       'agency_process_id', 'scc', 'reg_codes', 'reg_code_description',
       'process_description', 'reporting_period', 'emissions_operating_type',
       'calculation_parameter_value', 'calculation_parameter_uom',
       'calculation_material', 'calculation_parameter_type',
       'calc_data_source', 'calc_data_year', 'pollutant_code',
       'pollutant_desc', 'pollutant_type', 'total_emissions', 'emissions_uom',
       'emission_factor', 'ef_numerator_uom', 'ef_denominator_uom', 'ef_text',
    

In [41]:

df_TX_PM25 = df[sortby][["county","address","city","total_emissions","site_latitude","site_longitude"]]
df_TX_PM25.head

<bound method NDFrame.head of            county                              address          city  \
1621      Tarrant  IH 30 & OAKLAND BLVD; 2/3 MI ON OAK    FORT WORTH   
1633      Tarrant  IH 30 & OAKLAND BLVD; 2/3 MI ON OAK    FORT WORTH   
1657      Tarrant  FROM INTX OF HWY 121 & HADLEY EDERV   HALTOM CITY   
1672      Tarrant  FROM INTX OF HWY 121 & HADLEY EDERV   HALTOM CITY   
1698      Tarrant  X OF I 820 & FM1220; GO N 13M ; TUR    FORT WORTH   
...           ...                                  ...           ...   
4186129     Bexar             6055 W GREEN MOUNTAIN RD   SAN ANTONIO   
4186143    Harris                       2800 DECKER DR       BAYTOWN   
4186195     Ellis      2.5 M W OF MIDLOTHIAN ON HWY 67    MIDLOTHIAN   
4186205  Chambers                13330 HATCHERVILLE RD  MONT BELVIEU   
4186216     Brown               US HWY 377 (BRADY HWY)     BROWNWOOD   

         total_emissions  site_latitude  site_longitude  
1621              0.0030      32.769694      -9

In [42]:
# Distance Calculation in Excel
# = ACOS(COS(RADIANS(90-X1)) *COS(RADIANS(90-X2)) +SIN(RADIANS(90-X1)) *SIN(RADIANS(90-X2)) *COS(RADIANS(Y1-Y2)))*6371
# where: 
# X1 = Reference Latitude
# X2 = Row Latitude
# X1 = Reference Longitude
# X1 = Row Longitude
#
####### Note - Trig functions are in radians, so use "np.radians" as written in the above MS Excel function to convert to degrees.

def distance_lat_long(X1, X2, Y1, Y2):
    return np.arccos(np.cos(np.radians(90-X1)) * 
                    np.cos(np.radians(90-X2)) + 
                    np.sin(np.radians(90-X1)) *
                    np.sin(np.radians(90-X2)) *
                    np.cos(np.radians(Y1-Y2)))*6371

df_TX_PM25["Dist_Mon"] = distance_lat_long(
                Mon_Location[0],
                df_TX_PM25["site_latitude"],
                Mon_Location[1],
                df_TX_PM25["site_longitude"])

df_TX_PM25["Dist_Site"] = distance_lat_long(
                Site_Location[0],
                df_TX_PM25["site_latitude"],
                Site_Location[1],
                df_TX_PM25["site_longitude"])


In [43]:
# x = df_TX_PM25.groupby("total_emissions")

# df_final = df_TX_PM25[df_TX_PM25["Dist_Mon"] <= search_radius]
Mon_Total = df_TX_PM25[df_TX_PM25["Dist_Mon"] <= search_radius]["total_emissions"].sum()

Site_Total = df_TX_PM25[df_TX_PM25["Dist_Site"] <= search_radius]["total_emissions"].sum()

print("Total PM2.5 Emissions Surrounding Monitor: ",Mon_Total)
print("Total PM2.5 Emissions Surrounding Facility: ",Site_Total)
# .apply(lambda y: y[y["Dist_Mon"] <= search_radius]["total_emissions"].sum())

Total PM2.5 Emissions Surrounding Monitor:  34.238497943199995
Total PM2.5 Emissions Surrounding Facility:  182.7575982183
